In [1]:
import atlite
import cartopy.io.shapereader as shpreader
import geopandas as gpd
from shapely.geometry import box
import pandas as pd
import logging
from argparse import ArgumentParser
from datetime import datetime

In [2]:
print(f'0 : {datetime.now()}')

year = '2011'

logging.basicConfig(level=logging.INFO)

world = gpd.read_file("./data/ne_110m_admin_0_countries.shp")

# Drop uninhabited regions and Antarctica
world = world[(world.POP_EST > 0) & (world.NAME != "Antarctica")]

region = world
region_name = "world"

# Loop over the years
logging.info(f"Processing {year}")

INFO:root:Processing 2011


0 : 2024-10-05 20:33:38.057097


In [3]:
print(f'1 : {datetime.now()}')

# Define the cutout; this will not yet trigger any major operations
cutout = atlite.Cutout(
    path=f"{region_name}-{year}_timeseries", module="era5", 
    bounds=region.unary_union.bounds, 
    time=f"{year}",
    chunks={"time": 100,},)

/tmp/ipykernel_310884/4125285822.py:6: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  bounds=region.unary_union.bounds,
INFO:atlite.cutout:Building new cutout world-2011_timeseries.nc


1 : 2024-10-05 20:33:40.760429


In [4]:
print(cutout)

<Cutout "world-2011_timeseries">
 x = -180.00 ⟷ 179.75, dx = 0.25
 y = -55.50 ⟷ 83.50, dy = 0.25
 time = 2011-01-01 ⟷ 2011-12-31, dt = h
 module = era5
 prepared_features = []


In [5]:
# This is where all the work happens (this can take some time).
cutout.prepare(
    compression={"zlib": True, "complevel": 9},
    monthly_requests=True,
    concurrent_requests=True)

print(f'2 : {datetime.now()}')

INFO:atlite.data:Storing temporary files in /tmp/tmpvfv8rjvg
INFO:atlite.data:Calculating and writing with module era5:
INFO:atlite.datasets.era5:Requesting data for feature runoff...
INFO:atlite.datasets.era5:Requesting data for feature height...
INFO:atlite.datasets.era5:Requesting data for feature wind...
INFO:atlite.datasets.era5:Requesting data for feature influx...
/usr/local/python/3.12.1/lib/python3.12/site-packages/cads_api_client/legacy_api_client.py:101: UserWarning: This is a beta version. The following parameters have not been implemented yet: {'info_callback': <bound method Logger.debug of <Logger atlite.datasets.era5 (INFO)>>}.
  warnings.warn(


--------------------1--------------------
--------------------2--------------------
--------------------3--------------------
--------------------4--------------------
--------------------4.1--------------------
--------------------4.2--------------------
--------------------4.3--------------------
--------------------4.4--------------------
--------------------4.3--------------------
--------------------4.4--------------------
--------------------4.3--------------------
--------------------4.4--------------------
--------------------4.3--------------------
--------------------4.4--------------------
--------------------4.3--------------------
--------------------4.4--------------------
--------------------4.5--------------------
--------------------4.5.1--------------------
--------------------4.5.2--------------------
--------------------4.5.3--------------------
--------------------4.5.10.1--------------------
--------------------4.5.10.25--------------------
--------------------4.5

2024-10-05 20:33:47,403 INFO [2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
INFO:cads_api_client.processing:[2024-09-28T00:00:00] **Welcome to the New Climate Data Store (CDS)!** This new system is in its early days of full operations and still undergoing enhancements and fine tuning. Some disruptions are to be expected. Your 
[feedback](https://jira.ecmwf.int/plugins/servlet/desk/portal/1/create/202) is key to improve the user experience on the new CDS for the benefit of everyone. Thank you.
2024-10-05 20:33:47,404 WARNING [2024-09-26T00:00:00] Should you have not yet migrated from the old CDS system to the new CDS, please check our [informative pa

KeyboardInterrupt: 

In [ ]:
# Extract the wind power generation capacity factors
wind_power_generation = cutout.wind(
    "Vestas_V112_3MW", 
    capacity_factor_timeseries=True,
    )

print(f'3 : {datetime.now()}')